### 1. Nhn 뉴스 제목 검색
* requests, beautifulsoup 사용
* css selector - 속성 선택자 사용

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [3]:
url = 'https://news.naver.com/'
response = requests.get(url)
print('status code :' ,response.status_code)
print('response header : ', response.headers)
response.headers['content-type']

status code : 200
response header :  {'date': 'Sun, 26 Jul 2020 05:26:08 GMT', 'cache-control': 'no-cache', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'set-cookie': 'JSESSIONID=481AB5D0EEAFEF3AD2FA9050A90A9C9F; Path=/main; HttpOnly', 'content-language': 'ko-KR', 'vary': 'Accept-Encoding', 'content-encoding': 'gzip', 'transfer-encoding': 'chunked', 'content-type': 'text/html;charset=EUC-KR', 'referrer-policy': 'unsafe-url', 'server': 'nfront'}


'text/html;charset=EUC-KR'

In [4]:
# 응답 데이터 html 텍스트
html = response.text

In [5]:
# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')
tag_list = soup.select("a[href*=read.nhn]")
print(type(tag_list), len(tag_list))
for idx, a_tag in enumerate(tag_list):
    #print(a_tag)
    title = a_tag.text.strip()
    link = urljoin(url, a_tag['href'])
    print(idx, title, link)
    print('-------')

SelectorSyntaxError: Malformed attribute selector at position 1
  line 1:
a[href*=read.nhn]
 ^ (<string>)

### 2. Nhn 번역서비스 Papago 사용하기
* urllib 를 사용하기
* requests를 사용하기

In [6]:
# 네이버 Papago NMT API 예제
import os
import sys
import urllib.request

client_id = "Zo1vTslyKxKYskf_qfUk"
client_secret = "X5kGpbWEbv"

encText = urllib.parse.quote("Yesterday all my troubles seemed so far away.")
# query string
data = "source=en&target=ko&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"message":{"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0","result":{"srcLangType":"en","tarLangType":"ko","translatedText":"어제 나의 모든 고민들은 너무나 멀리 떨어져 있는 것 같았다.","engineType":"N2MT","pivot":null}}}


#### requests 예제로 변환하기 

In [ ]:
import requests

client_id = "Zo1vTslyKxKYskf_qfUk"
client_secret = "X5kGpbWEbv"
url = "https://openapi.naver.com/v1/papago/n2mt"
encText = "Yesterday all my troubles seemed so far away."

req_headers = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}
params = {
    'source':'en',
    'target':'ja',
    'text':encText
}
response = requests.post(url, headers=req_headers, data=params)
print('응답헤더값들',response.headers)
print('요청헤더값들',response.request.headers)
print('status code ',response.status_code)

if response.status_code == 200:
    print(response.text)
else:
    print('Error Code ',response.status_code)

In [ ]:
myjson = response.json()
print(type(myjson), myjson)

In [ ]:
myjson['message']['result']['translatedText']

### requests의 Request, Session 객체를 사용하는 방식으로 변환

In [7]:
import requests
# Request와 Session 클래스를 import
from requests import Request, Session

client_id = "Zo1vTslyKxKYskf_qfUk"
client_secret = "X5kGpbWEbv"
url = "https://openapi.naver.com/v1/papago/n2mt"
encText = "Yesterday all my troubles seemed so far away."

req_headers = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}
params = {
    'source':'en',
    'target':'ko',
    'text':encText
}

#Session 객체 생성
session = Session()
#Request 객체 생성
request = Request('POST', url, headers=req_headers, data=params)
# request의 prepare() 함수 호출
prepped = request.prepare()

# session의 send() 함수를 호출해서 서버에 요청을 전달
response = session.send(prepped)

print('응답헤더값들',response.headers)
print('요청헤더값들',response.request.headers)
print('status code ',response.status_code)

if response.status_code == 200:
    print(response.json()['message']['result']['translatedText'])
else:
    print('Error Code ',response.status_code)

응답헤더값들 {'Server': 'nginx', 'Date': 'Sun, 26 Jul 2020 05:56:39 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '271', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=5', 'Vary': 'Accept-Encoding', 'X-QUOTA': '45'}
요청헤더값들 {'X-Naver-Client-Id': 'Zo1vTslyKxKYskf_qfUk', 'X-Naver-Client-Secret': 'X5kGpbWEbv', 'Content-Length': '70', 'Content-Type': 'application/x-www-form-urlencoded'}
status code  200
어제 나의 모든 고민들은 너무나 멀리 떨어져 있는 것 같았다.
